**The Cutting Stock Problem** has been intensively researched due to its high importance in several industrial processes. It consists of finding an optimized way of cutting objects of known dimension into smaller items in order to meet a given demand. In general, the objective to be optimized is related to the minimization of the waste of material.

In [9]:
import random as rn
import re
import numpy as np
import math
from numpy import random

The effectiveness of a genetic algorithm for solving CSP depends on the choice of parameters such as the population size, selection method, crossover operator, and mutation rate. Careful tuning of these parameters is necessary to achieve good performance. Additionally, some modifications to the basic genetic algorithm can be made to improve its performance, such as using local search methods to improve the quality of the individuals in the population.

The goal of the CSP is to minimize the waste and sebsequently number of rolls, so a natural choice for the fitness function is to caculate the number of rolls based on the sum of th pattern of the chromosome.

The requests array remains unchanged, while the chromosome is composed of the index for each element in the req array. Mutation, crossover, and other operations are performed on these indices. Whenever necessary, we refer back to the original values in the requests array.

The GeneticAlgorithm class has 8 methods:

4. generate_solution(): In the first step, a solution need to be generated. this solution is the same *requests* list, except that the members are shuffled.
5. generate_neighbours(sol): For each solution number of *n_neighbours* are generated by swaping 2 elements of the solution. This way the neighbours are slightly different from solution and have a new pattern.


1. read_file: the input file is stored in variable *inp* then the string is splitted and only the digits remain. We turn the characters to integers in order to easily calculate cost in the next fucntion.

2. crossover: Child1 inherits the first x elements of Parent1’s pattern, and Child2 inherits the first x elements of Parent2’s pattern. The remainder of each child chromosome is filled according to the patterns and sequences observed in both parents. When an index from Parent2 is not present in Child1, it is appended to the end of Child1’s chromosome; if the index is already present, we proceed to the next element in Parent2’s chromosome.

3. fitness: For the solution *chromosome*, the sum of elements form i = 0 to i = length_os_solution is calculated. during calculation if the *curr_sum* exceeds the stock_length, number of rolls is incremented. The output is a percentage of optimal solution. If the cost is 100% it means the desired solution is achieved, if cost < 100% the number of rolls is higher than the desired answer and if cost > 100% we have reached a optimal solution.

4. mutation: The mutation is introduced by swapping two random indeces in a chromosome.

5. swap: a utility function to swap two elements in list *ls*

6. tornoment_selection: each time a number of tornoment_size chromosomes are chosen form the population to participate in the tornoment. The one chromosome with maximum fitness can proceed to the next generation. Again tornoment_size random chromosomes are chosen and the above loop is repeated.

7. generate_population: The chromosomes generated by generate_chromosome function are added to the population of desired size.

8. generate_chromosome: The first chromosome is generated by randomly shuffling the request array.

In [10]:
class GeneticAlgorithm():
    def __init__(self, file_name):
        self.stock_len, self.target, self.requests = self.read_file(file_name)
        self.n_req = len(self.requests)
        self.population_size = math.ceil(self.stock_len * 3/4)
        self.crossover_rate = 0.9
        self.mutation_rate = 0.1
        self.tournament_size = math.ceil(self.population_size * 1/3)

    # Read input files
    def read_file(self, file_name):
        f = open(file_name)
        inp = f.read()
        f.close()
        # Split digits
        req = re.split('\D+', inp)
        req = req[1:-1]
        req = list(map(int, req))
        stock_len = req[0]
        ans = req[-1]
        req = req[1:-1]
        return stock_len, ans, req

    # Generate the first pattern
    def generate_chromosome(self):
        chromosome = []
        for i in range(self.n_req):
            chromosome.append(i)
        random.shuffle(chromosome)
        return chromosome

    # Generate the population of chromosomes, each have length of the request input
    def generate_population(self):
        population = []
        for _ in range(self.population_size):
            population.append(self.generate_chromosome())
        return population

    # Evaluate each chromosome based on rolls they use
    def fitness(self, chromosome):
        n_rolls = 0
        curr_sum = 0
        for i in range(self.n_req):
            curr_sum += self.requests[chromosome[i]]
            if curr_sum > self.stock_len:
                n_rolls += 1
                curr_sum = self.requests[chromosome[i]]
        return self.target/n_rolls*100

    # Utility fucntion
    def swap(self, i, j, ls):
        tmp = ls[i]
        ls[i] = ls[j]
        ls[j] = tmp
        return ls

    # Introduce mutation by swapping two random places in a chromosome
    def mutation(self, ch):
        for _ in range(int(self.mutation_rate * len(ch))):
            ch = self.swap(random.randint(0, len(ch)), random.randint(0, len(ch)), ch)
        return ch

    # Introduce crossing over by mixing parent 1 and parent 2
    def crossover(self, p1, p2):
        co_point = random.randint(len(p1))
        # The first co_point of child1 = parent1
        child1 = p1[:co_point]
        # The first co_point of child2 = parent2
        child2 = p2[:co_point]
        # The rest of child1 is filled with parent2 pattern if the element is not already present in child1
        for i in p2:
            try:
                child1.index(i)
            except ValueError:
                child1.append(i)
        # The rest of child2 is filled with parent1 pattern if the element is not already present in child2
        for i in p1:
            try:
                child2.index(i)
            except ValueError:
                child2.append(i)
        return child1, child2

    # Each time a random chromosome is added to tournament and only the chromosome with highest fitness level will make it to the next generation
    def tournament_selection(self, pop):
        next_gen = []

        for _ in range(len(pop) - 1):
            tournament = []
            maxfit = 0;
            maxfit_index = 0
            for i in range(self.tournament_size):
                x = pop[random.randint(0,len(pop))]
                tournament.append(x)
                x_fitness = self.fitness(x)
                if x_fitness > maxfit:
                    maxfit = x_fitness
                    maxfit_index = i
            next_gen.append(tournament[maxfit_index])
        return next_gen



In [13]:
def optimizer(file_name):
    GA = GeneticAlgorithm(file_name)
    population = GA.generate_population()

    best_solution = None
    best_fitness = 0
    max_gen = 10
    least_rolls = 0

    for generation in range(max_gen + 1):
        # Check for new best solution in the population
        for i in population:
            if GA.fitness(i) > best_fitness:
                best_fitness = GA.fitness(i)
                best_solution = i
                least_rolls = GA.target/best_fitness*100

        # Selection
        selected = GA.tournament_selection(population)
        # Ensuring max is always selected for the next generation
        selected.append(i)


        # Introduction of crossover and mutation to the population
        offspring = []
        while len(offspring) < GA.population_size:
            if random.random() < GA.crossover_rate:
                parent1 = selected[random.randint(0,len(selected))]
                parent2 = selected[random.randint(0,len(selected))]
                child1, child2 = GA.crossover(parent1, parent2)
                offspring.append(GA.mutation(child1))
                offspring.append(GA.mutation(child2))
            else:
                offspring.extend(selected[:2])

        # Update population
        population = offspring

        if generation % 1 == 0:
            print(f"Generation No. {generation} - Best Fitness(%) {best_fitness} - Rolls No. {least_rolls}")
        # If the desired result is reached, break and done!
        if (best_fitness >= 100):
            break


In [10]:
optimizer("input1.stock")

Generation No. 0 - Best Fitness(%) 89.47368421052632 - Rolls No. 56.99999999999999
Generation No. 10 - Best Fitness(%) 92.72727272727272 - Rolls No. 55.00000000000001
Generation No. 20 - Best Fitness(%) 94.44444444444444 - Rolls No. 54.0
Generation No. 30 - Best Fitness(%) 94.44444444444444 - Rolls No. 54.0
Generation No. 40 - Best Fitness(%) 94.44444444444444 - Rolls No. 54.0
Generation No. 50 - Best Fitness(%) 94.44444444444444 - Rolls No. 54.0
Generation No. 60 - Best Fitness(%) 94.44444444444444 - Rolls No. 54.0
Generation No. 70 - Best Fitness(%) 94.44444444444444 - Rolls No. 54.0
Generation No. 80 - Best Fitness(%) 94.44444444444444 - Rolls No. 54.0
Generation No. 90 - Best Fitness(%) 94.44444444444444 - Rolls No. 54.0
Generation No. 100 - Best Fitness(%) 94.44444444444444 - Rolls No. 54.0


In [ ]:
optimizer("input2.stock")

Generation No. 0 - Best Fitness(%) 86.90476190476191 - Rolls No. 84.0


In [5]:
optimizer("input3.stock")

Generation No. 0 - Best Fitness(%) 106.4814814814815 - Rolls No. 107.99999999999999


In [24]:
optimizer("input4.stock")

Generation No. 0 - Best Fitness(%) 95.1417004048583 - Rolls No. 247.00000000000003
Generation No. 50 - Best Fitness(%) 97.5103734439834 - Rolls No. 241.0
Generation No. 100 - Best Fitness(%) 97.5103734439834 - Rolls No. 241.0
Generation No. 150 - Best Fitness(%) 98.32635983263597 - Rolls No. 239.0
Generation No. 200 - Best Fitness(%) 98.32635983263597 - Rolls No. 239.0
Generation No. 250 - Best Fitness(%) 98.32635983263597 - Rolls No. 239.0
Generation No. 300 - Best Fitness(%) 98.32635983263597 - Rolls No. 239.0
Generation No. 350 - Best Fitness(%) 98.32635983263597 - Rolls No. 239.0
Generation No. 400 - Best Fitness(%) 98.32635983263597 - Rolls No. 239.0
Generation No. 450 - Best Fitness(%) 98.32635983263597 - Rolls No. 239.0
Generation No. 500 - Best Fitness(%) 98.32635983263597 - Rolls No. 239.0
